In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from tqdm import tnrange, tqdm_notebook
from time import sleep
import pickle



In [2]:
test_string="test_"  # or ""

path="../Data/WoS_data/Disambiguated_authors/"

list_years=[2014,2015]
list_df=[]
for year in list_years:
    
    df_aux= pd.read_table('../Data/WoS_data/Disambiguated_authors/'+test_string+str(year)+'.txt',header=None,names=['uid','author_id','author_name','affiliation','idx'])
    list_df.append(df_aux)
    
# concatenate two dataframes with exactly the same column structure
df = pd.concat(list_df)




100000

In [3]:
# unpack each row into as many rows as authors in the paper:

def process(row):
    
    aids = [int(a) for a in row.author_id.split('|')] # list of author ids in that row
    n = len(aids)
    names = row.author_name.split('|')  # list of author names in that row
    affil = []
    if pd.isnull(row.affiliation):   # if there is no affiliation info, then affil is a list of n nans
        affil = [np.nan]*n
    else:
        affil_list = row.affiliation.split('|') # if there is affiliation info, i build a list of affiliations                       
        
        for idx in row.idx.split('|'): ## list of affiliations indexes for each author , idx is a list of affil indexes for one author                                                           
            
            
            if idx=='-1':
                affil.append('?')
                # OR:
                # affil.append(row.affiliation)
            else:
                try:
                    affil.append('|'.join([affil_list[int(i)] for i in idx.split(',')]))
                except IndexError: 
                    affil.append('?')

                   
    return pd.DataFrame({'uid':[row.uid]*n,'author_id':aids,'author_name':names,'affiliation':affil,'seq':range(len(aids))})


result = pd.concat([process(row[1]) for row in  df.iterrows() ] )   # row is a tuple row_index, the entire row info (searchable by:  row[1].author_id  )
#result = pd.concat([process(row[1]) for row in  tqdm_notebook(df.iterrows(), desc='1st loop')   ]     )   # row is a tuple row_index, the entire row info (searchable by:  row[1].author_id  )



done


In [4]:
# get rid of all rows without a valid author_id
result=result.loc[result['author_id'] != -1]  # i filter out the authors without desambiguated author id
# i reasign the variable results so i dont eat up all RAM
#df2.head()


486260


In [5]:
# i filter out non-USA authors 

result.affiliation.shape, result.affiliation.dropna().shape

result=result.dropna(subset=['affiliation']).loc[result['affiliation'].dropna().str.contains('USA')]  

103241


In [6]:
# count number of publications

pub_counts = result.author_id.value_counts()  # it returns a series of index, values  (the values column will have the name of the original column it comes from)
pub_counts.name = 'tot_n_pubs'  # i change the name of the value column of the series
FINAL=result.join(pub_counts,on='author_id') #join: matching INDEX from pub_counts series with author_id column from df2 

FINAL.sort_values("author_id").head(200)
# for 2 dataframes, use merge: df2.merge(df2,on='uid')
#grouped.head(5)


#count_per_author = grouped.Author_ID.value_counts()

#author_data = grouped.groupby('Author_ID').apply(lambda x: x.Author_name.unique())

#author_data[author_data.apply(lambda x: len(x))>1].shape
          #joined = author_data.join(counts_per_author,on='Author_ID')
#new_df.head()
print len(FINAL)

103241


In [37]:
    
def re_capilatizing_lastnames(authorname):
    
    author_lastname=authorname.split(",")[0].replace("-"," ").replace("'"," ").capitalize().replace(" ","-").replace(" ","'")
    try:
         rest_name=authorname.split(",")[1].strip()#.replace("-"," ").replace("'"," ").capitalize().replace(" ","-").replace(" ","'")
   
         new_name= author_lastname+", "+rest_name
             
    except: 
        return authorname                    
    
    return new_name

%time result = FINAL.author_name.apply(re_capilatizing_lastnames)
#%time result = test(10000)

#%timeit 2+2


CPU times: user 28 ms, sys: 0 ns, total: 28 ms
Wall time: 23.1 ms
100000000 loops, best of 3: 15 ns per loop


In [38]:
unique_author_names = FINAL.groupby('author_id').apply(lambda x: x.author_name.unique())

multiple_author_names=unique_author_names[unique_author_names.apply(lambda x: len(x)>1)]
#len(unique_autaffiliationhor_names[unique_author_names.apply(lambda x: len(x)>1)])


len(unique_author_names)



92386

In [40]:
# i write the lookup file for authors with multiple name spellings 

ini_year=str(list_years[0])
final_year=str(list_years[-1])
# FINAL.to_csv(path+"final_"+ini_year+"-"+final_year+".csv", sep='\t')

multiple_author_names.to_csv(path+"lookup_multiple_author_names_"+ini_year+"-"+final_year+".csv", sep='\t')



In [16]:
# i build a master dict


print len(FINAL)
    
master_dict={}

for row in FINAL.iterrows():
    author_id=row[1].author_id
    
    affil=row[1].affiliation
    author_name=row[1].author_name
    paper_id=row[1].uid
    tot_num_pub=row[1].tot_n_pubs
      
    author_lastname = author_name.split(',')[0].capitalize()
        
        
    
    try:
        master_dict[author_id]
    except KeyError:        
        master_dict[author_id]={}
    
    
    
        
    try:
        master_dict[author_id]['affiliation'].append(affil)
    except KeyError:
        master_dict[author_id]['affiliation']=[]
        master_dict[author_id]['affiliation'].append(affil)
        
           
    try:
        master_dict[author_id]['author_name'].append(author_name)
    except KeyError:
        master_dict[author_id]['author_name']=[]
        master_dict[author_id]['author_name'].append(author_name)
        
             
    try:
        master_dict[author_id]['author_lastname'].append(author_lastname)
    except KeyError:
        master_dict[author_id]['author_lastname']=[]
        master_dict[author_id]['author_lastname'].append(author_lastname)
        
        
              
    try:
        master_dict[author_id]['paper_id'].append(paper_id)
    except KeyError:
        master_dict[author_id]['paper_id']=[]
        master_dict[author_id]['paper_id'].append(paper_id)
        
        
               
    master_dict[author_id]['tot_num_pub']=tot_num_pub
        
        
        
        
        
  
print len(master_dict)

103241
92386


In [17]:
# i pickle the master dict
cont_multiple_lastnames_per_id=0.
for author_id in sorted(master_dict):
    master_dict[author_id]['author_lastname']= list(set(master_dict[author_id]['author_lastname']))
    
    if len(master_dict[author_id]['author_lastname']) >1:
        cont_multiple_lastnames_per_id +=1.
        
        
# #     if len(master_dict[author_id]['affiliation'])  >1:
#          print author_id
#          for llave in master_dict[author_id]:              
#                  print llave,": ", master_dict[author_id][llave]
            #x=pd.Series([1,2,3])
#x.name='pop'
#x
            
#          raw_input()


print "num authors in dict:", len(master_dict), "   fraction of conflicting diff. lastnames with same id:", cont_multiple_lastnames_per_id/len(master_dict)

filehandler = open( path+"disambig_authors_USA_"+ini_year+"-"+final_year+".pickle","wb")
pickle.dump(master_dict,filehandler)




num authors in dict: 92386    fraction of conflicting diff. lastnames with same id: 0.000292252072825


In [19]:
FINAL.sort_values("author_id").head(10)

,affiliation,author_id,author_name,seq,uid,tot_n_pubs
4,"Baylor Coll Med, Houston, TX 77030 USA",96,"Aagaard, Kjersti",4,WOS:000361140900151,3
10,"Baylor Coll Med, Div Maternal Fetal Med, Dept ...",96,"Aagaard, Kjersti M.",10,WOS:000337503800033,3
6,"Baylor Coll Med, Houston, TX 77030 USA",96,"Aagaard, Kjersti",6,WOS:000361140900081,3
0,"Baylor Coll Med, Div Maternal Fetal Med, Houst...",106,"Aagaard, Kjersti",0,WOS:000330322600522,1
3,"AbbVie Inc, N Chicago, IL USA",149,"Anderson, Jaclyn K.",3,WOS:000344384901102,1
0,"Boston Univ, Sch Med, Boston Med Ctr, Boston, ...",187,"Abdullah, Abu S.",0,WOS:000339040600081,1
1,"Duke Univ, Dept Med, Duke Canc Inst, Div Hemat...",478,"Allen, Jessica L.",1,WOS:000357284300020,1
1,"Amer Assisted Living Nurses Assoc, Napa, CA 94...",713,"Allen, Josh",1,WOS:000331595600017,1
1,"US Army Res Lab, Adelphi, MD 20783 USA",733,"Allen, J. L.",1,WOS:000340321300134,1
2,"Univ Minnesota, Sch Med, Duluth, MN 55812 USA",743,"Allen, James",2,WOS:000361758100009,1


In [55]:
def grouping(input_df):
    
    
    tot_n_pubs=input_df.tot_n_pubs.iloc[0]
 
    result = pd.Series({'author_names':input_df.author_name.unique(),'affiliations':input_df.affiliation.unique(),\
                         'seqs':input_df.seq.values,'tot_n_pub':tot_n_pubs}) #if the values of some of the columns are list, use SERIES instead of DATAFRAME            
    
    return result
 

grouped = FINAL.groupby('author_id').apply(grouping)



,affiliations,author_names,seqs,tot_n_pub
author_id,,,,
2568,"[Amgen Inc, Med Chem, Thousand Oaks, CA 91320 ...","[Allen, John G.]",[6],1
7997,"[Oregon Hlth & Sci Univ, Dept Behav Neurosci, ...","[Abraham, Antony D.]",[0],1
11048,"[CALTECH, Div Biol & Biol Engn, Pasadena, CA 9...","[Anderson, David J.]",[3],2
11343,"[Weill Cornell Med Coll, CADB, White Plains, N...","[Anderson, Deborah K.]",[0],1
13733,"[Boston Childrens Hosp, Inst Professionalism &...","[Brown, Stephen D.]",[4],1
14898,"[Oregon State Univ, Sch Civil & Construct Engn...","[Brown, Shane]",[0],1
16762,"[Univ New Mexico, Albuquerque, NM 87131 USA]","[Abraham, Shirley]",[0],1
25783,"[Columbia Univ, Coll Phys & Surg, Dept Med, Ne...","[Campbell, Julie]",[1],1
30677,"[Washington State Univ, Sch Civil & Environm E...","[Adam, Jennifer]",[3],1


In [ ]:
# i write the final file with all needed info on authors
grouped.to_csv(path+"final_"+ini_year+"-"+final_year+".csv", sep='\t')



In [43]:
FINAL.head().groupby('author_id').count().reset_index()

,author_id,affiliation,author_name,seq,uid,tot_n_pubs
0,5174335,1,1,1,1,1
1,8517127,1,1,1,1,1
2,15148332,1,1,1,1,1
3,31793454,1,1,1,1,1
4,35445596,1,1,1,1,1
